# Machine Translation Project (English to French)

In [31]:
import collections
import numpy as np
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, GRU, LSTM, Bidirectional, Dropout, Activation, TimeDistributed, RepeatVector
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [32]:
def load_data(path):
    input_file = path
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

english_sentences = load_data('/kaggle/input/language/english.txt')
french_sentences = load_data('/kaggle/input/language/french.txt')

In [33]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

### Preprocess

In [35]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [36]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

In [37]:
def preprocess(x,y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [38]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

### Model 1: RNN

In [39]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    #Hyperparameters
    learning_rate = 0.005
    
    # Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

#Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.4586 - loss: 2.6638 - val_accuracy: 0.6358 - val_loss: nan
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.6314 - loss: 1.2838 - val_accuracy: 0.6729 - val_loss: nan
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.6647 - loss: 1.1131 - val_accuracy: 0.6931 - val_loss: nan
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.6822 - loss: 1.0124 - val_accuracy: 0.7073 - val_loss: nan
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.6998 - loss: 0.9385 - val_accuracy: 0.7317 - val_loss: nan
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.7154 - loss: 0.8766 - val_accuracy: 0.7467 - val_loss: nan
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.7262 - loss: 0.8357 - val_accuracy: 0.7641 - val_loss: nan
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.7407 - loss: 0.7831 - val_accuracy: 0.7544 - val_loss: nan
E

In [40]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
new jersey est parfois calme en mois de mai et il est neigeux en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### Model 2: Bidirectional RNNs

In [41]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    #Hyperparameters
    learning_rate = 0.005
    
    # Build the layers
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(bd_rnn_model.summary())

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 21, 256)        │       100,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_10             │ (None, 21, 1024)       │       263,168 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 21, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_11             │ (None, 21, 344)        │       352,600 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 716,376 (2.73 MB)

 Trainable params: 716,376 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.5047 - loss: 2.4165 - val_accuracy: 0.6497 - val_loss: nan
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.6482 - loss: 1.2076 - val_accuracy: 0.6796 - val_loss: nan
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.6747 - loss: 1.0604 - val_accuracy: 0.6956 - val_loss: nan
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.6863 - loss: 0.9803 - val_accuracy: 0.7039 - val_loss: nan
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.6961 - loss: 0.9208 - val_accuracy: 0.7151 - val_loss: nan
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.7050 - loss: 0.8754 - val_accuracy: 0.7237 - val_loss: nan
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.7125 - loss: 0.8382 - val_accuracy: 0.7303 - val_loss: nan
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.7177 - loss: 0.8139 - val_accuracy: 0.7346 - 

In [42]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
new jersey est parfois parfois en l' et il et neigeux est en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


### Model 3: Embedding

In [43]:
def bidirectional_embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    # Hyperparameters
    learning_rate = 0.005
    
    # Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    
    # Compile model
    model.compile(loss = sparse_categorical_crossentropy,
                  optimizer = Adam(learning_rate),
                  metrics = ['accuracy'])
    
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Build the model
embed_rnn_model = bidirectional_embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(embed_rnn_model.summary())

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
    

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 21, 256)        │        50,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 21, 512)        │       789,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_12             │ (None, 21, 1024)       │       525,312 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 21, 1024)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_13             │ (None, 21, 344)        │       352,600 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,718,360 (6.56 MB)

 Trainable params: 1,718,360 (6.56 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.5958 - loss: 2.0945 - val_accuracy: 0.8797 - val_loss: nan
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.8904 - loss: 0.3487 - val_accuracy: 0.9382 - val_loss: nan
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9390 - loss: 0.1956 - val_accuracy: 0.9588 - val_loss: nan
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9572 - loss: 0.1380 - val_accuracy: 0.9683 - val_loss: nan
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9671 - loss: 0.1061 - val_accuracy: 0.9729 - val_loss: nan
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9714 - loss: 0.0919 - val_accuracy: 0.9769 - val_loss: nan
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9758 - loss: 0.0784 - val_accuracy: 0.9785 - val_loss: nan
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 90ms/step - accuracy: 0.9777 - loss: 0.0721 - val_accuracy: 0.

In [44]:
# Print prediction(s)
print("Prediciton:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print('\nOriginal text:')
print(english_sentences[:1])

Prediciton:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


In [46]:
embed_rnn_model.save('english_to_french_model.h5')
# Serialize English Tokenizer to JSON
with open('english_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(english_tokenizer.to_json(), ensure_ascii=False))
    
# Serialize French Tokenizer to JSON
with open('french_tokenizer.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(french_tokenizer.to_json(), ensure_ascii=False))
    
# Save max lengths
max_french_sequence_length_json = max_french_sequence_length
with open('sequence_length.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(max_french_sequence_length_json, ensure_ascii=False))